# 模型融合

最后一章，做模型融合，这应该是所有比赛最后都需要经历的一环，因为模型融合确实可以提高一点比赛分数，如视频直播中所说，可以榨干模型的最后一点能力。对有的任务，模型融合提升还是蛮大的，如回归类问题。而分类问题，模型融合带来的提升貌似比较小，最高可以带来2%左右的提高。

### 交叉验证训练多模型
深度学习训练一般需要耗费很长时间，如果算力限制，可以考虑留出法训练多个模型，这次模型融合，以交叉验证训练十个模型，然后取均值，或者投票法，可以达到模型融合的效果。

### 测试数据扩展TTA
第二种是，在测试时做测试数据扩展，即在测试时对测试数据进行轻微调整，然后进行多次预测，最后投票。

### 一个训练模型的多个局部最优点模型融合
其实单个模型也可以做模型融合，这里介绍了一种叫Snapshot Ensembles的方法。即在周期性变大变小的学习率中，checkpoint，取得许多局部最优点，然后对着一系列checkpoint点取模型参数，也可以获得融合的模型。


在交叉验证时，常常加入Dropout，以增强模型随机性。

In [1]:
from torch.utils.data.dataset import Dataset
import os, sys, glob, shutil, json
import cv2

from PIL import Image
import numpy as np

from tqdm import tqdm, tqdm_notebook

import torch
#torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable   

In [2]:
#定义自己的数据集类,从Dataset处继承
class MyDataset(Dataset):
    #定义初始化方法
    def __init__(self,img_path,img_label,transform=None):
        #三个参数，图片路径，图片标签和转化方式，转化方式与数据集扩展有关
        self.img_path=img_path
        self.img_label=img_label
        #注意我们可以先不进行数据增强，所以要加判断
        if transform !=None:
            self.transform=transform
        else:
            self.transform=None
    
    #构建数据集需要两个方法，一个是getitem,一个是len,这里我觉得getitem就是调用这个datasset后，会一个一个返回数据和标签的方法
    #len会返回数据集大小，这个getitem是按编号返回数据及标签。
    
    #这里的open是打开一个文件夹，注意，这里文件是一个一个读取的，img_path是一个list!所以有索引！！！，同理，label——path
    #也是一样.
    def __getitem__(self, index):
        #读取图片的方法可以换
        img = Image.open(self.img_path[index]).convert('RGB')
    #转换img
        if self.transform is not None:
            img = self.transform(img)
        
        # 设置最长的字符长度为5个,读取label,注意这里转换了一下数据类型
        lbl = np.array(self.img_label[index], dtype=np.int)
        #由于是固定长度，所以将不足5位的地方补足了
        lbl = list(lbl)  + (5 - len(lbl)) * [10]
        return img, torch.from_numpy(np.array(lbl[:5]))
   #这里再次注意img_path是一个列表
    def __len__(self):
        return len(self.img_path)

In [3]:
#glob.glob读取结果是一个列表，类似于linnix的查找文件方式，可迭代对象
train_path = glob.glob('E:/Machine Learning/CV字符/mchar_train/mchar_train/*.png')
#列表排序
train_path.sort()
#json.load()是用来读取文件的，即，将文件打开然后就可以直接读取。示例如下
#with open("文件名") as f:
#    result=json.load(f)
train_json = json.load(open('E:/Machine Learning/CV字符/mchar_train.json'))
#json文件格式类似于字典，下面是一个试例
#{"000000.png": {"height": [219, 219], "label": [1, 9], "left": [246, 323], "top": [77, 81], "width": [81, 96]},
train_label = [train_json[x]['label'] for x in train_json]
print(len(train_path), len(train_label))

train_loader = torch.utils.data.DataLoader(
    MyDataset(train_path, train_label,
                transforms.Compose([
                    transforms.Resize((64, 128)),
                    transforms.RandomCrop((60, 120)),
                    transforms.ColorJitter(0.3, 0.3, 0.2),
                    transforms.RandomRotation(5),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])), 
    batch_size=40, 
    shuffle=True, 
    num_workers=0,
)


30000 30000


In [4]:
val_path = glob.glob('E:/Machine Learning/CV字符/mchar_val/mchar_val/*.png')
val_path.sort()
val_json = json.load(open('E:/Machine Learning/CV字符/mchar_val.json'))
val_label = [val_json[x]['label'] for x in val_json]
print(len(val_path), len(val_label))

val_loader = torch.utils.data.DataLoader(
    MyDataset(val_path, val_label,
                transforms.Compose([
                    #transforms.Resize((60, 120)),
                    # transforms.ColorJitter(0.3, 0.3, 0.2),
                    # transforms.RandomRotation(5),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])), 
    batch_size=40, 
    shuffle=False, 
    num_workers=0,
)

10000 10000


In [ ]:
class SVHN_Model1(nn.Module):
    def __init__(self):
        super(SVHN_Model1, self).__init__()
        # CNN提取特征模块
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2)),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2)),
            nn.ReLU(), 
            nn.Dropout(0.25),
            nn.MaxPool2d(2),
        )
        # 
        self.fc1 = nn.Linear(32*3*7, 11)
        self.fc2 = nn.Linear(32*3*7, 11)
        self.fc3 = nn.Linear(32*3*7, 11)
        self.fc4 = nn.Linear(32*3*7, 11)
        self.fc5 = nn.Linear(32*3*7, 11)
        self.fc6 = nn.Linear(32*3*7, 11)
    
    def forward(self, img):
        feat = self.cnn(img)
        feat = feat.view(feat.shape[0], -1)
        c1 = self.fc1(feat)
        c2 = self.fc2(feat)
        c3 = self.fc3(feat)
        c4 = self.fc4(feat)
        c5 = self.fc5(feat)
        c6 = self.fc6(feat)
        return c1, c2, c3, c4, c5, c6

## TTA代码
测试集数据扩增（Test Time Augmentation，简称TTA）也是常用的集成学习技巧，数据扩增不仅可以在训练时候用，而且可以同样在预测时候进行数据扩增，对同一个样本预测三次，然后对三次结果进行平均。

In [ ]:
 def predict(test_loader, model, tta=10):
    model.eval()
    test_pred_tta = None
    # TTA 次数
    for _ in range(tta):
        test_pred = []
    
        with torch.no_grad():
            for i, (input, target) in enumerate(test_loader):
                c0, c1, c2, c3, c4, c5 = model(data[0])
                output = np.concatenate([c0.data.numpy(), c1.data.numpy(),
                   c2.data.numpy(), c3.data.numpy(),
                   c4.data.numpy(), c5.data.numpy()], axis=1)
                test_pred.append(output)
        
        test_pred = np.vstack(test_pred)
        if test_pred_tta is None:
            test_pred_tta = test_pred
        else:
            test_pred_tta += test_pred
    
    return test_pred_tta

## 小结

- 集成学习只能在一定程度上提高精度，并需要耗费较大的训练时间，因此建议先使用提高单个模型的精度，再考虑集成学习过程；
- 具体的集成学习方法需要与验证集划分方法结合，Dropout和TTA在所有场景有可以起作用